### Note: I received four participant CSV files and determined that the second and third file were the same participant, so I removed participant_3.csv from the analysis

### Import necessary packages

In [1]:
import pandas as pd
import numpy as np

## Read each participant's CSV file to their own dataframe

In [2]:
p1 = pd.read_csv('participant_1.csv')
p2 = pd.read_csv('participant_2.csv')
p3 = pd.read_csv('participant_4.csv')

### Check shape of dataframes to see the number of datapoints before cleaning

In [3]:
p1.shape[0]+p2.shape[0]+p3.shape[0]

94373

## Remove null values and missing data

In [4]:
p1temporary = p1.replace(0,np.nan)

In [5]:
p2temporary = p2.replace(0,np.nan)

In [6]:
p3temporary = p3.replace(0,np.nan)

In [7]:
p1_clean = p1temporary.dropna(subset=['r_size','r_conf','r_x_pos','r_y_pos','l_size','l_conf','l_x_pos','l_y_pos'])

In [8]:
p2_clean = p2temporary.dropna(subset=['r_size','r_conf','r_x_pos','r_y_pos','l_size','l_conf','l_x_pos','l_y_pos'])

In [9]:
p3_clean = p3temporary.dropna(subset=['r_size','r_conf','r_x_pos','r_y_pos','l_size','l_conf','l_x_pos','l_y_pos'])

### Fix timestamp at 0 seconds

In [10]:
p1_clean.at[0,'timestamp']=0

In [11]:
p2_clean.at[0,'timestamp']=0

In [12]:
p3_clean.at[0,'timestamp']=0

### Check shape of file post-cleaning to make sure there are still enough datapoints

In [13]:
p1_clean.shape[0]+p2_clean.shape[0]+p3_clean.shape[0]

88507

Only ~6000 datapoints (~6%) were lost by removing zeros from the data; probably worth seeing how many more datapoints are lost due to a threshhold for left/right eye confidence

## Insert a confidence threshhold

In [14]:
p1_temp = p1_clean[p1_clean['r_conf'] > 0.25]

In [15]:
p1_corr = p1_temp[p1_temp['l_conf'] > 0.25].reset_index(drop=True)

In [16]:
p2_temp = p2_clean[p2_clean['r_conf'] > 0.25]

In [17]:
p2_corr = p2_temp[p2_temp['l_conf'] > 0.25].reset_index(drop=True)

In [18]:
p3_temp = p3_clean[p3_clean['r_conf'] > 0.25]

In [19]:
p3_corr = p3_temp[p3_temp['l_conf'] > 0.25].reset_index(drop=True)

In [20]:
p1_corr.shape[0]+p2_corr.shape[0]+p3_corr.shape[0]

83810

Requiring each eye to have a minimum confidence value of 0.25 removes a mere 4700 datapoints from the original 95000, yet likely confers a benefit from removing outliers.

I also noticed some of the pupil sizes were odd (a few were the value of Pi for example) so I will create another threshhold.

## Insert a pupil size threshhold

In [21]:
p1_pupil = p1_corr[p1_corr['r_size'] > 5]

In [22]:
p1_final = p1_pupil[p1_pupil['l_size'] > 5]

In [23]:
p2_pupil = p2_corr[p2_corr['r_size'] > 5]

In [24]:
p2_final = p2_pupil[p2_pupil['l_size'] > 5]

In [25]:
p3_pupil = p3_corr[p3_corr['r_size'] > 5]

In [26]:
p3_final= p3_pupil[p3_pupil['l_size'] > 5]

In [27]:
p1_final.shape[0]+p2_final.shape[0]+p3_final.shape[0]

83810

It seems the previous threshhold for confidence was enough to take care of pupil size outliers.

## Merge the participant dataframes into a single dataframe

In [28]:
frames = [p1_final, p2_final, p3_final]

In [29]:
part = pd.concat(frames, keys=['Participant 1', 'Participant 2', 'Participant 3'])

#### random shit testing hierarchy for multiindex

In [30]:
participants = pd.concat([part, part], keys=['Experiment Data'], axis=1)

In [31]:
participants

Experiment Data                                    \
                          timestamp        r_size    r_conf   r_x_pos   
Participant 1 0               0.000     80.530000  0.940600   302.805   
              1               0.116     82.526900  0.947200   302.112   
              2               0.132     83.706400  0.941765   302.712   
              3               0.168     83.795700  0.946122   254.151   
              4               0.201     84.952300  0.940600   263.454   
              5               0.216     84.053500  0.947200   302.817   
              6               0.337     84.478900  0.946122   301.612   
              7               0.385     84.623200  0.940600   301.170   
              8               0.400     84.140800  0.940600   300.214   
              9               0.452     83.843800  0.939388   328.365   
              10              0.468     83.822100  0.940600   304.063   
              11              0.484     83.543200  0.940600   304.700   
              12              0.604     83.712700  0.940600   305.609   
              13              0.652     83.522900  0.939388   305.214   
              14              0.672     83.241300  0.940600   304.890   
              15              0.704     82.708100  0.941765   346.798   
              16              0.720     82.871000  0.940600   304.678   
              17              0.804     82.180600  0.940600   304.748   
              18              0.840     82.235500  0.947200   308.966   
              19              0.956     81.308300  0.947200   309.772   
              20              1.040     80.103000  0.947200   192.357   
              21              1.092     78.860400  0.952857   309.197   
              22              1.156     78.317200  0.947200   309.649   
              23              1.192     78.554800  0.940600   309.653   
              24              1.260     77.349400  0.940600   300.890   
              25              1.344     76.998300  0.947200   300.517   
              26              1.460     77.145500  0.940600   343.589   
              27              1.476     77.330200  0.934000   341.771   
              28              1.560     77.322100  0.940600   305.607   
              29              1.828     74.957600  0.946122   303.058   
...                             ...           ...       ...       ...   
Participant 3 15393         563.351  15165.770922  0.472222   356.873   
              15394         563.387  13462.213394  0.676471   366.527   
              15395         563.419  12539.024379  0.735294   359.928   
              15396         564.895   9467.515413  0.823529   452.957   
              15397         565.063  30326.360630  0.500000   535.285   
              15398         565.265  10521.141006  0.794118  1269.800   
              15399         565.699  13512.604097  0.666667   361.884   
              15400         565.799  11986.539640  0.818182   373.457   
              15401         565.867  13528.720966  0.685714   352.578   
              15402         565.899  14069.035749  0.600000   359.760   
              15403         565.999  11391.575054  0.781250   371.055   
              15404         566.067  11354.677758  0.787879   371.589   
              15405         566.135  20437.033101  0.514286   353.865   
              15406         566.203  11836.041278  0.818182   365.027   
              15407         566.267  34918.338423  0.388889   379.824   
              15408         566.671   1701.618020  0.294118   322.182   
              15409         566.703  13590.391539  0.611111   320.476   
              15410         566.739  14139.257888  0.514286   335.910   
              15411         567.395  11243.593196  0.727273   370.571   
              15412         567.507  11699.488428  0.800000   354.237   
              15413         567.577   7256.245767  0.747431   419.327   
              15414         568.211  10568.427014  0.657143   126.417   
     

## Perform basic data analysis

### Group by block number and type

In [32]:
part

timestamp        r_size    r_conf   r_x_pos      r_y_pos  \
Participant 1 0          0.000     80.530000  0.940600   302.805   158.092000   
              1          0.116     82.526900  0.947200   302.112   159.561000   
              2          0.132     83.706400  0.941765   302.712   160.014000   
              3          0.168     83.795700  0.946122   254.151  -138.958000   
              4          0.201     84.952300  0.940600   263.454  -155.037000   
              5          0.216     84.053500  0.947200   302.817   161.019000   
              6          0.337     84.478900  0.946122   301.612   161.122000   
              7          0.385     84.623200  0.940600   301.170   160.773000   
              8          0.400     84.140800  0.940600   300.214   161.264000   
              9          0.452     83.843800  0.939388   328.365   190.017000   
              10         0.468     83.822100  0.940600   304.063   152.556000   
              11         0.484     83.543200  0.940600   304.700   152.408000   
              12         0.604     83.712700  0.940600   305.609   150.464000   
              13         0.652     83.522900  0.939388   305.214   149.857000   
              14         0.672     83.241300  0.940600   304.890   150.052000   
              15         0.704     82.708100  0.941765   346.798   -83.851600   
              16         0.720     82.871000  0.940600   304.678   149.907000   
              17         0.804     82.180600  0.940600   304.748   151.008000   
              18         0.840     82.235500  0.947200   308.966   159.152000   
              19         0.956     81.308300  0.947200   309.772   158.947000   
              20         1.040     80.103000  0.947200   192.357  -173.397000   
              21         1.092     78.860400  0.952857   309.197   160.343000   
              22         1.156     78.317200  0.947200   309.649   161.007000   
              23         1.192     78.554800  0.940600   309.653   160.019000   
              24         1.260     77.349400  0.940600   300.890   161.014000   
              25         1.344     76.998300  0.947200   300.517   160.674000   
              26         1.460     77.145500  0.940600   343.589   -71.049700   
              27         1.476     77.330200  0.934000   341.771   -95.046400   
              28         1.560     77.322100  0.940600   305.607   158.510000   
              29         1.828     74.957600  0.946122   303.058   148.997000   
...                        ...           ...       ...       ...          ...   
Participant 3 15393    563.351  15165.770922  0.472222   356.873   131.998000   
              15394    563.387  13462.213394  0.676471   366.527    32.155000   
              15395    563.419  12539.024379  0.735294   359.928     0.410618   
              15396    564.895   9467.515413  0.823529   452.957  -110.704000   
              15397    565.063  30326.360630  0.500000   535.285  -175.450000   
              15398    565.265  10521.141006  0.794118  1269.800  3131.290000   
              15399    565.699  13512.604097  0.666667   361.884    36.253400   
              15400    565.799  11986.539640  0.818182   373.457     6.527330   
              15401    565.867  13528.720966  0.685714   352.578    42.958900   
              15402    565.899  14069.035749  0.600000   359.760   111.354000   
              15403    565.999  11391.575054  0.781250   371.055    -1.055790   
              15404    566.067  11354.677758  0.787879   371.589     0.435867   
              15405    566.135  20437.033101  0.514286   353.865   112.139000   
              15406    566.203  11836.041278  0.818182   365.027    11.648600   
              15407    566.267  34918.338423  0.388889   379.824   116.015000   
              15408    566.671   1701.618020  0.294118   322.182   120.174000   
              15409    566.703  13590.391539  0.611111   320.476   121.098000   
              15410    566.739  14139.257888  0.5142

In [33]:
p1_blocknum = part.loc['Participant 1'].groupby(['block_number', 'block_type'], as_index = False)

In [34]:
p2_blocknum = part.loc['Participant 2'].groupby(['block_number', 'block_type'], as_index = False)

## Preliminary data analysis for participants 1 and 2

In [35]:
p1_analysis = p1_blocknum['r_size','r_conf','r_x_pos','r_y_pos','l_size','l_conf','l_x_pos','l_y_pos'].describe()
p1_analysis

r_size                                                                    \
    count        mean        std      min        25%        50%         75%   
0   200.0   75.119788   1.534451  72.5879  73.939775   74.63810   76.303475   
1   432.0   81.517885  10.616744  30.7274  75.523350   81.96940   84.437275   
2   625.0   77.298602   2.750458  56.4005  75.035300   77.50660   79.452400   
3    84.0  109.344910  66.105883  74.6420  76.076975   78.10700   84.619550   
4   188.0   78.865911   2.201778  74.7267  77.550250   78.63180   79.766425   
5   357.0   78.568041   3.421106  40.1633  77.047800   78.59410   80.154900   
6   233.0   74.736615   1.796160  70.5051  73.893700   74.76510   75.562600   
7   346.0   81.716425  13.622830  27.7907  77.546775   81.66275   84.507225   
8    57.0   80.431168   2.654904  75.4359  77.607000   81.52120   82.397600   
9   129.0   78.921981   5.605579  72.7842  75.370900   78.76170   80.246800   
10  219.0   78.292483   2.712160  72.9418  76.106000   78.64750   80.438000   
11  376.0   74.725645   2.853876  51.4010  72.608625   75.17820   76.773425   
12  222.0   71.557097   2.367572  65.7816  70.586225   72.05570   73.501850   
13  389.0   77.249710   2.440658  58.3322  75.513100   76.98090   79.148200   
14  296.0   85.260654  23.831953  32.4377  74.217925   78.33640   87.031550   
15  193.0   76.799670   1.760255  61.3347  75.959900   76.76680   77.523400   
16  451.0   82.071559   6.587969  43.8933  76.391900   80.77720   87.620500   
17  447.0   83.955536   6.049879  73.4876  79.905400   82.61870   87.973700   
18  185.0   75.944581   2.230909  72.1401  74.094500   75.65520   77.960400   
19  366.0   83.762189  12.354398  67.2629  77.353125   80.47435   87.640925   
20  282.0   87.167748   8.763637  68.1462  79.438925   86.56740   93.597850   
21  221.0   74.332635   1.672470  68.6766  73.555200   74.31510   75.422800   
22  420.0   75.179333   4.160575  68.0680  71.737525   75.02525   78.611000   
23  401.0   76.436210   7.296741  67.3667  72.464000   76.81400   78.651300   
24  238.0   74.409642   1.741419  70.0336  73.354075   74.77830   75.699500   
25  344.0   82.505729  26.862381  70.7852  74.302350   76.23630   78.920600   
26  337.0   78.041997   6.397455  36.2836  76.626900   78.20750   80.425700   
27  192.0   79.986336   5.927628  70.4267  76.189550   78.63595   83.471000   
28  330.0   77.439783   4.622362  71.4407  74.756575   76.53230   78.910775   
29  391.0   76.525133   5.857811  72.0564  74.298800   76.07660   77.197750   
..    ...         ...        ...      ...        ...        ...         ...   
62  548.0   74.872091   2.481969  68.8620  73.183550   74.65000   76.408375   
63  232.0   72.471700   2.539169  65.6097  70.406900   71.85020   74.804800   
64  462.0   73.651826   4.847016  66.9834  70.481750   72.54930   76.301675   
65  260.0   76.404256   3.364465  70.5016  74.684650   75.85485   77.207150   
66  238.0   76.367476   1.939443  72.1501  75.536525   76.30905   77.387575   
67  457.0   74.362579   1.900712  65.5086  72.923300   74.39130   75.579000   
68  175.0   74.145177   3.740778  54.6001  72.631850   74.66440   76.818800   
69  231.0   70.795605   3.946278  62.2438  68.158450   70.60570   72.277100   
70  395.0   74.489902   4.153869  45.1650  71.905950   74.26370   77.279600   
71  209.0  162.757443  77.597535  69.7272  74.248600  223.55300  230.307000   
72  168.0   80.491065  12.522954  67.1332  78.241300   79.05920   79.924100   
73  322.0  161.220165  76.972197  66.3193  80.002175  228.40000  234.237750   
74  602.0   77.652960   5.883655  68.0343  74.359850   77.35235   80.445750   
75  209.0   76.866341  16.316969  71.0739  74.028000   74.71340   75.659300   
76  406.0   77.325180   3.811664  72.6800  74.786750   76.30275   78.856475   
77  272.0   79.884321   2.780948  50.1597  79.209175   80.40640   81.411825   
78  238.0   74.254544   1.638263  71.1160  73.247800   74.04510   74.908625   
79  467.0   77.279831  19.119270  71.5015  74.015650 

In [36]:
p2_analysis = p2_blocknum['r_size','r_conf','r_x_pos','r_y_pos','l_size','l_conf','l_x_pos','l_y_pos'].describe()
p2_analysis

r_size                                                                      \
    count        mean        std       min         25%        50%         75%   
0   218.0  103.893111   5.005330   96.3320   98.897750  104.01650  108.224250   
1   465.0   96.261304  10.075218   81.8861   85.875900   96.15760  102.682000   
2   765.0  117.363801   8.495578   96.3905  112.856000  119.41600  122.624000   
3   232.0  102.217900   5.640954   54.5974   99.163350  101.38450  106.027500   
4   460.0  107.914500   6.755175   94.6174  103.240250  107.41550  111.239750   
5   516.0  120.093529  10.805415  108.0980  115.031000  118.95550  122.766000   
6   232.0  102.091268   8.236619   87.3244   94.313050  104.96150  109.854000   
7   469.0  105.127349   9.717891   74.8186   97.682900  102.96100  110.001000   
8   495.0  108.038330   7.517870   89.4264  101.022000  107.83300  113.239500   
9   231.0   97.434909   8.897610   82.1637   91.424650   97.06740  105.075500   
10  470.0  106.093923   9.339488   92.0977  100.732500  103.67400  107.877750   
11  516.0  112.171213  12.244693   31.7863  103.925250  114.30450  120.101500   
12  228.0   93.779242   5.088607   86.1784   88.148125   94.17585   97.563800   
13  468.0  104.283360   6.538192   90.3375   99.264100  103.85150  106.196750   
14  699.0  122.767775   7.520357  105.5950  119.011500  121.57100  124.471500   
15  227.0  107.031996   5.854537   65.7960  102.261500  107.08300  112.382000   
16  470.0  103.703084   4.587326   88.0600  100.287500  104.12750  107.542000   
17  697.0  118.717658  13.784135   34.0684  112.506000  119.30700  125.268000   
18  224.0   88.677578   7.601165   70.9496   81.029600   88.89240   94.933450   
19  476.0   96.932150   9.381542   75.6696   89.686500   97.66495  103.282500   
20  683.0  108.563252   4.916539   98.6193  104.442500  108.47200  112.545000   
21  232.0   88.527410   8.619794   75.2502   80.778725   88.47035   95.961100   
22  450.0  109.658407   8.510561   53.5505  102.462500  109.33500  116.761250   
23  695.0  106.501670   7.090325   75.9766  100.969000  106.89700  113.008000   
24  231.0   91.583291   3.640427   85.6577   88.710750   90.71040   94.695200   
25  462.0   98.672778  10.715426   63.1869   86.386150  102.19650  105.800750   
26  890.0  109.965309   6.934521   87.9099  105.326000  111.43400  114.864500   
27  218.0   94.155069   6.948120   68.8673   88.360875   92.94210  100.800500   
28  468.0   99.982239   4.547025   90.5489   97.963675   99.63475  103.627500   
29  684.0  112.663993   7.502302   98.6700  108.269750  112.16350  116.966250   
..    ...         ...        ...       ...         ...        ...         ...   
62  599.0  116.184376   8.628554   53.3177  111.723000  114.71500  119.576000   
63  221.0  102.273042   6.409100   82.3825   95.922700  102.92800  107.948000   
64  472.0  102.005643   4.668742   89.6670   98.775000  102.43450  104.561500   
65  486.0  110.779377   5.443912   96.1029  108.233500  111.23000  114.295500   
66  219.0   97.030934   6.338199   47.7841   93.419850   97.29250  100.315500   
67  464.0   96.143357   6.556793   84.0490   91.291300   94.58615  101.433250   
68  432.0   98.708525   5.580869   87.6316   93.634925   99.77655  103.293500   
69  228.0   93.910490   5.383626   83.9068   87.946900   95.15800   98.514925   
70  468.0   92.142990   4.313597   83.1237   88.719650   91.94095   95.988225   
71  465.0   93.731964   7.012445   77.1428   92.032200   95.95820   98.649600   
72  218.0  103.113463  20.993418   47.3685   95.887500  102.43750  106.025000   
73  461.0  103.558648   3.403568   84.1364  101.700000  103.34300  105.458000   
74  553.0  110.342332   6.862384   97.4405  105.057000  110.53500  115.842000   
75  238.0  103.828075   4.432187   91.3258  101.018500  104.06800  107.425750   
76  446.0   91.766816   5.096200   60.3498   88.138725   90.47825   95.980850   
77  490.0   99.018887  30.520579   49.8837   87.616925   95.44305  101.617250   
78  225.0   85.368934   7.272810  

### Participant 3 has no data for block number or type, so I cannot group the eye data by question

In [37]:
p3_describe = part.loc['Participant 3'].describe()

In [38]:
interim = pd.concat([p3_describe, p3_describe], keys=['Participant 3'])
interim

timestamp        r_size        r_conf       r_x_pos  \
Participant 3 count  15423.000000  15423.000000  15423.000000  15423.000000   
              mean     285.559568   2852.633492      0.959265    355.483903   
              std      163.199200   4509.885676      0.066277     47.894952   
              min        0.000000    857.124210      0.264706    108.849000   
              25%      140.083000   1410.812084      0.956957    318.419500   
              50%      291.427000   1638.301183      0.970000    358.046000   
              75%      426.619000   1938.099744      1.000000    387.100000   
              max      569.619000  56062.410156      1.000000   1269.800000   

                          r_y_pos         l_size        l_conf       l_x_pos  \
Participant 3 count  15423.000000   15423.000000  15423.000000  15423.000000   
              mean      72.265906   10908.206851      0.910927    355.483903   
              std       62.061069    6327.209585      0.042625     47.894952   
              min     -451.685000    1544.691925      0.257143    108.849000   
              25%       44.917850    9024.120915      0.904375    318.419500   
              50%       60.251600   10263.304281      0.913224    358.046000   
              75%      100.428000   11761.513862      0.920800    387.100000   
              max     4273.340000  178922.711408      1.000000   1269.800000   

                          l_y_pos  block_number  difficulty  
Participant 3 count  15423.000000           0.0         0.0  
              mean      72.265906           NaN         NaN  
              std       62.061069           NaN         NaN  
              min     -451.685000           NaN         NaN  
              25%       44.917850           NaN         NaN  
              50%       60.251600           NaN         NaN  
              75%      100.428000           NaN         NaN  
              max     4273.340000           NaN         NaN

In [39]:
p3_analysis = pd.concat([interim,interim], keys=['Processed'], axis=1)
p3_analysis

Processed                                            \
                        timestamp        r_size        r_conf       r_x_pos   
Participant 3 count  15423.000000  15423.000000  15423.000000  15423.000000   
              mean     285.559568   2852.633492      0.959265    355.483903   
              std      163.199200   4509.885676      0.066277     47.894952   
              min        0.000000    857.124210      0.264706    108.849000   
              25%      140.083000   1410.812084      0.956957    318.419500   
              50%      291.427000   1638.301183      0.970000    358.046000   
              75%      426.619000   1938.099744      1.000000    387.100000   
              max      569.619000  56062.410156      1.000000   1269.800000   

                                                                              \
                          r_y_pos         l_size        l_conf       l_x_pos   
Participant 3 count  15423.000000   15423.000000  15423.000000  15423.000000   
              mean      72.265906   10908.206851      0.910927    355.483903   
              std       62.061069    6327.209585      0.042625     47.894952   
              min     -451.685000    1544.691925      0.257143    108.849000   
              25%       44.917850    9024.120915      0.904375    318.419500   
              50%       60.251600   10263.304281      0.913224    358.046000   
              75%      100.428000   11761.513862      0.920800    387.100000   
              max     4273.340000  178922.711408      1.000000   1269.800000   

                                                           
                          l_y_pos block_number difficulty  
Participant 3 count  15423.000000          0.0        0.0  
              mean      72.265906          NaN        NaN  
              std       62.061069          NaN        NaN  
              min     -451.685000          NaN        NaN  
              25%       44.917850          NaN        NaN  
              50%       60.251600          NaN        NaN  
              75%      100.428000          NaN        NaN  
              max     4273.340000          NaN        NaN

### MERGE?????!?!?

In [40]:
analysis = pd.concat([p1_analysis, p2_analysis], keys=['Participant 1','Participant 2'])
analysis

r_size                                               \
                  count        mean        std       min         25%   
Participant 1 0   200.0   75.119788   1.534451   72.5879   73.939775   
              1   432.0   81.517885  10.616744   30.7274   75.523350   
              2   625.0   77.298602   2.750458   56.4005   75.035300   
              3    84.0  109.344910  66.105883   74.6420   76.076975   
              4   188.0   78.865911   2.201778   74.7267   77.550250   
              5   357.0   78.568041   3.421106   40.1633   77.047800   
              6   233.0   74.736615   1.796160   70.5051   73.893700   
              7   346.0   81.716425  13.622830   27.7907   77.546775   
              8    57.0   80.431168   2.654904   75.4359   77.607000   
              9   129.0   78.921981   5.605579   72.7842   75.370900   
              10  219.0   78.292483   2.712160   72.9418   76.106000   
              11  376.0   74.725645   2.853876   51.4010   72.608625   
              12  222.0   71.557097   2.367572   65.7816   70.586225   
              13  389.0   77.249710   2.440658   58.3322   75.513100   
              14  296.0   85.260654  23.831953   32.4377   74.217925   
              15  193.0   76.799670   1.760255   61.3347   75.959900   
              16  451.0   82.071559   6.587969   43.8933   76.391900   
              17  447.0   83.955536   6.049879   73.4876   79.905400   
              18  185.0   75.944581   2.230909   72.1401   74.094500   
              19  366.0   83.762189  12.354398   67.2629   77.353125   
              20  282.0   87.167748   8.763637   68.1462   79.438925   
              21  221.0   74.332635   1.672470   68.6766   73.555200   
              22  420.0   75.179333   4.160575   68.0680   71.737525   
              23  401.0   76.436210   7.296741   67.3667   72.464000   
              24  238.0   74.409642   1.741419   70.0336   73.354075   
              25  344.0   82.505729  26.862381   70.7852   74.302350   
              26  337.0   78.041997   6.397455   36.2836   76.626900   
              27  192.0   79.986336   5.927628   70.4267   76.189550   
              28  330.0   77.439783   4.622362   71.4407   74.756575   
              29  391.0   76.525133   5.857811   72.0564   74.298800   
...                 ...         ...        ...       ...         ...   
Participant 2 62  599.0  116.184376   8.628554   53.3177  111.723000   
              63  221.0  102.273042   6.409100   82.3825   95.922700   
              64  472.0  102.005643   4.668742   89.6670   98.775000   
              65  486.0  110.779377   5.443912   96.1029  108.233500   
              66  219.0   97.030934   6.338199   47.7841   93.419850   
              67  464.0   96.143357   6.556793   84.0490   91.291300   
              68  432.0   98.708525   5.580869   87.6316   93.634925   
              69  228.0   93.910490   5.383626   83.9068   87.946900   
              70  468.0   92.142990   4.313597   83.1237   88.719650   
              71  465.0   93.731964   7.012445   77.1428   92.032200   
              72  218.0  103.113463  20.993418   47.3685   95.887500   
              73  461.0  103.558648   3.403568   84.1364  101.700000   
              74  553.0  110.342332   6.862384   97.4405  105.057000   
              75  238.0  103.828075   4.432187   91.3258  101.018500   
              76  446.0   91.766816   5.096200   60.3498   88.138725   
              77  490.0   99.018887  30.520579   49.8837   87.616925   
              78  225.0   85.368934   7.272810   76.9997   81.409700   
              79  437.0   83.508964   5.904626   64.9972   77.866800   
              80  515.0   95.658547   6.541407   80.1700   91.991900   
              81  231.0   94.911455   6.007544   85.1225   89.159350   
              82  467.0   82.579121   6.691238   73.0680   76.931800   
              83  650.0  102.996926   9.180193   60.7863   98.201575   
              84  233.0   96.133678   6.737013   87.6641 

In [41]:
final = pd.merge(participants, analysis, how='left', left_index=True, right_index=True)
final

Experiment Data                                    \
                          timestamp        r_size    r_conf   r_x_pos   
Participant 1 0               0.000     80.530000  0.940600   302.805   
              1               0.116     82.526900  0.947200   302.112   
              2               0.132     83.706400  0.941765   302.712   
              3               0.168     83.795700  0.946122   254.151   
              4               0.201     84.952300  0.940600   263.454   
              5               0.216     84.053500  0.947200   302.817   
              6               0.337     84.478900  0.946122   301.612   
              7               0.385     84.623200  0.940600   301.170   
              8               0.400     84.140800  0.940600   300.214   
              9               0.452     83.843800  0.939388   328.365   
              10              0.468     83.822100  0.940600   304.063   
              11              0.484     83.543200  0.940600   304.700   
              12              0.604     83.712700  0.940600   305.609   
              13              0.652     83.522900  0.939388   305.214   
              14              0.672     83.241300  0.940600   304.890   
              15              0.704     82.708100  0.941765   346.798   
              16              0.720     82.871000  0.940600   304.678   
              17              0.804     82.180600  0.940600   304.748   
              18              0.840     82.235500  0.947200   308.966   
              19              0.956     81.308300  0.947200   309.772   
              20              1.040     80.103000  0.947200   192.357   
              21              1.092     78.860400  0.952857   309.197   
              22              1.156     78.317200  0.947200   309.649   
              23              1.192     78.554800  0.940600   309.653   
              24              1.260     77.349400  0.940600   300.890   
              25              1.344     76.998300  0.947200   300.517   
              26              1.460     77.145500  0.940600   343.589   
              27              1.476     77.330200  0.934000   341.771   
              28              1.560     77.322100  0.940600   305.607   
              29              1.828     74.957600  0.946122   303.058   
...                             ...           ...       ...       ...   
Participant 3 15393         563.351  15165.770922  0.472222   356.873   
              15394         563.387  13462.213394  0.676471   366.527   
              15395         563.419  12539.024379  0.735294   359.928   
              15396         564.895   9467.515413  0.823529   452.957   
              15397         565.063  30326.360630  0.500000   535.285   
              15398         565.265  10521.141006  0.794118  1269.800   
              15399         565.699  13512.604097  0.666667   361.884   
              15400         565.799  11986.539640  0.818182   373.457   
              15401         565.867  13528.720966  0.685714   352.578   
              15402         565.899  14069.035749  0.600000   359.760   
              15403         565.999  11391.575054  0.781250   371.055   
              15404         566.067  11354.677758  0.787879   371.589   
              15405         566.135  20437.033101  0.514286   353.865   
              15406         566.203  11836.041278  0.818182   365.027   
              15407         566.267  34918.338423  0.388889   379.824   
              15408         566.671   1701.618020  0.294118   322.182   
              15409         566.703  13590.391539  0.611111   320.476   
              15410         566.739  14139.257888  0.514286   335.910   
              15411         567.395  11243.593196  0.727273   370.571   
              15412         567.507  11699.488428  0.800000   354.237   
              15413         567.577   7256.245767  0.747431   419.327   
              15414         568.211  10568.427014  0.657143   126.417   
     

In [47]:
test = pd.merge(final, interim, how='left', left_index=True, right_index=True)
test

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


(Experiment Data, timestamp)  (Experiment Data, r_size)  \
Participant 1 0                             0.000                  80.530000   
              1                             0.116                  82.526900   
              2                             0.132                  83.706400   
              3                             0.168                  83.795700   
              4                             0.201                  84.952300   
              5                             0.216                  84.053500   
              6                             0.337                  84.478900   
              7                             0.385                  84.623200   
              8                             0.400                  84.140800   
              9                             0.452                  83.843800   
              10                            0.468                  83.822100   
              11                            0.484                  83.543200   
              12                            0.604                  83.712700   
              13                            0.652                  83.522900   
              14                            0.672                  83.241300   
              15                            0.704                  82.708100   
              16                            0.720                  82.871000   
              17                            0.804                  82.180600   
              18                            0.840                  82.235500   
              19                            0.956                  81.308300   
              20                            1.040                  80.103000   
              21                            1.092                  78.860400   
              22                            1.156                  78.317200   
              23                            1.192                  78.554800   
              24                            1.260                  77.349400   
              25                            1.344                  76.998300   
              26                            1.460                  77.145500   
              27                            1.476                  77.330200   
              28                            1.560                  77.322100   
              29                            1.828                  74.957600   
...                                           ...                        ...   
Participant 3 15393                       563.351               15165.770922   
              15394                       563.387               13462.213394   
              15395                       563.419               12539.024379   
              15396                       564.895                9467.515413   
              15397                       565.063               30326.360630   
              15398                       565.265               10521.141006   
              15399                       565.699               13512.604097   
              15400                       565.799               11986.539640   
              15401                       565.867               13528.720966   
              15402                       565.899               14069.035749   
              15403                       565.999               11391.575054   
              15404                       566.067               11354.677758   
              15405                       566.135               20437.033101   
              15406                       566.203               11836.041278   
              15407                       566.267               34918.338423   
              15408                       566.671                1701.618020   
              15409                       566.703               13590.391539   
              15410                       566.739               14139.257888   
              15411  

In [48]:
test.loc['Participant 3']

,"(Experiment Data, timestamp)","(Experiment Data, r_size)","(Experiment Data, r_conf)","(Experiment Data, r_x_pos)","(Experiment Data, r_y_pos)","(Experiment Data, l_size)","(Experiment Data, l_conf)","(Experiment Data, l_x_pos)","(Experiment Data, l_y_pos)","(Experiment Data, block_number)",...,r_size,r_conf,r_x_pos,r_y_pos,l_size,l_conf,l_x_pos,l_y_pos,block_number,difficulty
0,0.000,2506.654815,0.885714,288.806,158.424000,12083.412115,0.914000,288.806,158.424000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.031,2478.761819,0.818182,288.195,158.247000,12065.299138,0.932653,288.195,158.247000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.066,1988.929900,1.000000,284.397,168.938000,11935.131846,0.934000,284.397,168.938000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.098,1960.245206,0.972500,287.666,162.834000,11929.400877,0.924286,287.666,162.834000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.130,2060.917457,1.000000,287.554,161.081000,12086.373807,0.934000,287.554,161.081000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.167,2069.649056,1.000000,285.203,162.887000,12309.889012,0.928824,285.203,162.887000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.199,1955.947147,0.972500,285.505,162.231000,12302.377988,0.927400,285.505,162.231000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.234,1950.933486,0.972500,285.850,161.609000,12412.321786,0.915111,285.850,161.609000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.267,1955.226037,0.972500,285.186,163.330000,12694.512773,0.917686,285.186,163.330000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.298,1978.209819,0.960400,284.116,164.258000,12999.106844,0.907971,284.116,164.258000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Export to CSV

In [44]:
final.to_csv('C:\\Users\\Zach\\Desktop\\Senseye Challenge\\processed_data\\Processed data copy2.csv')